In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import sys
sys.path.append('/content/gdrive/My Drive/workspace')

%cd '/content/gdrive/My Drive/workspace'

Mounted at /content/gdrive
/content/gdrive/My Drive/workspace


In [2]:
from PIL import Image

import numpy as np

from collections import Counter

import random

from itertools import permutations

from os.path import join
from os import listdir

from FaceRecognizer import FaceRecognizer

from scipy.spatial import distance
from scipy.optimize import brentq
from scipy.interpolate import interp1d

import dlib

from helpers import read_folds

from evaluation import plot_roc_curve, plot_score_distribution

from preprocessing import face_closeup
import preprocessing as prep

from matplotlib import pyplot as plt

import pandas as pd

from importlib import reload

from keras import backend as K

from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import Input, concatenate, Dense, Cropping2D, Flatten, Activation, Reshape

Using TensorFlow backend.


In [0]:
root_path = 'lfw'
predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector_path = 'mmod_human_face_detector.dat'
path_to_triplets = 'll2_ul1000_triplets_10_10.json'
path_to_weights = 'vgg_face_weights.h5'

path_to_save_weights = 'vgg_face_weights_finetune_last_layer.h5'

In [0]:
cnn_face_detector = dlib.cnn_face_detection_model_v1(detector_path)

sp = dlib.shape_predictor(predictor_path)

In [0]:
def triplet_loss(y_true, y_pred, alpha = 0.7):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """
    print('y_pred.shape = ',y_pred)
    
    total_length = y_pred.shape.as_list()[-1]
    if total_length is None:    
        total_lenght = 2622
        
    print(total_lenght)
    print()
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    print(pos_dist,neg_dist)
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

In [6]:
adam_optim = Adam(lr=0.00001)


Instructions for updating:
Colocations handled automatically by placer.


In [7]:
anchor_input = Input((224,224,3, ), name='anchor_input')
positive_input = Input((224,224,3, ), name='positive_input')
negative_input = Input((224,224,3, ), name='negative_input')

# Shared embedding layer for positive and negative items
vgg = FaceRecognizer(path_to_weights=path_to_weights)
vgg_model = vgg.get_vgg_model()

add_layer = False
if add_layer:
    new_model = Sequential()
    for layer in vgg_model.layers[:-2]:
        new_model.add(layer)

    for layer in new_model.layers:
        layer.trainable = False

    new_model.add(Dense(1000,name='feature_reduction_dense'))
    new_model.add(Reshape((1000,),name='last_flatened_layer'))
    Shared_DNN = new_model
else:
    for layer in vgg_model.layers:
        layer.trainable=False
    vgg_model.layers[-3].trainable=True
    
    Shared_DNN = vgg_model


encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.compile(loss=triplet_loss, optimizer=adam_optim)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
y_pred.shape =  Tensor("merged_layer/concat:0", shape=(?, ?), dtype=float32)
2622

Tensor("loss/merged_layer_loss/Sum:0", shape=(?,), dtype=float32) Tensor("loss/merged_layer_loss/Sum_1:0", shape=(?,), dtype=float32)


In [8]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 2622)         145002878   anchor_input[0][0]               
                                                                 positive_input[0][0]             
          

In [0]:
import json
with open(path_to_triplets, 'r') as f:
    triplets = json.load(f)


In [10]:
len(triplets)

73940

In [0]:
my_cache = {}
def f(name):
    name = '/' + name.rsplit('_',1)[0] + '/'+ name
    if name in  my_cache:
        return my_cache[name]
    else:
        img = dlib.load_rgb_image(root_path+name)
        funneled_image_arr = prep.cropped_and_aligned(img, cnn_face_detector, sp,method='cnn',size=224)
        
        my_cache[name] = funneled_image_arr
    
    return my_cache[name]

# def f(name):
#     name = '/' + name.rsplit('_',1)[0] + '/'+ name
#     img = dlib.load_rgb_image(root_path+name)
#     funneled_image_arr = prep.cropped_and_aligned(img, cnn_face_detector, sp,method='cnn',size=224)
#     return funneled_image_arr

In [0]:
def data_gen(triplets, batch_size):
    steps_per_epoch = len(triplets)//batch_size
    dummy_y = np.empty((batch_size,1))
    while True:
        next_anc = []
        next_pos = []
        next_neg = []
        for i in range(batch_size):
            a, p, n = triplets[i]
            next_anc.append(f(a))
            next_pos.append(f(p))
            next_neg.append(f(n))
        yield [np.array(next_anc), np.array(next_pos), np.array(next_neg)], dummy_y

In [0]:
batch_size = 64
data = triplets 

In [14]:
model.fit_generator(data_gen(data,batch_size), steps_per_epoch=len(data)//batch_size, epochs=2, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2


/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 2 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))
/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 3 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))


  26/1155 [..............................] - ETA: 56:59 - loss: 0.6009

KeyboardInterrupt: ignored

In [0]:
Shared_DNN.save_weights(path_to_save_weights)